<div style="font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
  <p style="font-size: 18px; font-weight: bold; color: #4A90E2;">Dev Vyas</p>
  <p style="font-size: 18px; color: #333;">Student ID: <span style="color: #D14F4F;">202201453</span></p>

  <p style="font-size: 18px; font-weight: bold; color: #4A90E2;">Dharmi Patel</p>
  <p style="font-size: 18px; color: #333;">Student ID: <span style="color: #D14F4F;">202201467</span></p>

  <p style="font-size: 18px; font-weight: bold; color: #4A90E2;">Preksha Shah</p>
  <p style="font-size: 18px; color: #333;">Student ID: <span style="color: #D14F4F;">202203004</span></p>
</div>


<div style="font-family: Arial, sans-serif; line-height: 1.6; color: #333; padding: 20px; background-color: #f4f4f9; border-radius: 8px; border: 1px solid #ddd;">
  <h2 style="color: #2c3e50; font-size: 24px;">Feature Selection and Model Fitting</h2>
  <p style="font-size: 16px; color: #555;">In this notebook, we perform <strong>feature selection</strong> and subsequently apply various <strong>machine learning models</strong> to train and make predictions on the dataset. The following steps are carried out:</p>
  <ul style="font-size: 16px; color: #555;">
    <li><strong>Feature Selection</strong>: We use several feature selection techniques like <strong>SelectKBest</strong>, <strong>RFE (Recursive Feature Elimination)</strong>, and <strong>Principal Component Analysis (PCA)</strong> to identify the most important features in the dataset.</li>
    <li><strong>Model Fitting</strong>: Once the relevant features are selected, we apply both <strong>regression</strong> (e.g., <strong>Linear Regression</strong>, <strong>XGBoost Regressor</strong>) and <strong>classification</strong> (e.g., <strong>Logistic Regression</strong>, <strong>Random Forest</strong>, <strong>XGBoost Classifier</strong>) models.</li>
    <li><strong>Model Evaluation</strong>: The performance of each model is assessed using various metrics such as <strong>accuracy</strong>, <strong>mean absolute error</strong>, <strong>mean squared error</strong>, <strong>R2 score</strong>, and <strong>ROC AUC score</strong>.</li>
  </ul>
  <p style="font-size: 16px; color: #555;">This workflow enables us to understand the significance of different features and how well the models can predict the target variable.</p>
</div>


In [30]:
# Importing necessary libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, roc_auc_score, classification_report
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier


<div style="font-family: Arial, sans-serif; line-height: 1.6; color: #333; padding: 20px; background-color: #f4f4f9; border-radius: 8px; border: 1px solid #ddd;">
  <h2 style="color: #2c3e50; font-size: 24px;">Reading the Excel File: energy.xlsx</h2>
  <p style="font-size: 16px; color: #555;">In this section, we will load the <strong>energy.xlsx</strong> file into a pandas DataFrame for further analysis. Here's how to do it:</p>
  <ul style="font-size: 16px; color: #555;">
    <li><strong>Step 1:</strong> Upload the Excel file into the environment.</li>
    <li><strong>Step 2:</strong> Use the <code>pd.read_excel()</code> function to read the contents of the file into a pandas DataFrame.</li>
    <li><strong>Step 3:</strong> Display the first few rows using <code>energy.head()</code> to inspect the data.</li>
  </ul>
  
</div>


In [31]:
energy = pd.read_excel('energy.xlsx')

In [32]:
energy = energy.drop(index=0).reset_index(drop=True)

In [33]:
energy.columns

Index(['Month', 'Total Fossil Fuels Production',
       'Nuclear Electric Power Production',
       'Total Renewable Energy Production', 'Total Primary Energy Production',
       'Primary Energy Imports', 'Primary Energy Exports',
       'Primary Energy Net Imports', 'Primary Energy Stock Change and Other',
       'Total Fossil Fuels Consumption', 'Nuclear Electric Power Consumption',
       'Total Renewable Energy Consumption',
       'Total Primary Energy Consumption'],
      dtype='object')

In [34]:
energy.dtypes

,0
Month,datetime64[ns]
Total Fossil Fuels Production,float64
Nuclear Electric Power Production,float64
Total Renewable Energy Production,float64
Total Primary Energy Production,float64
Primary Energy Imports,float64
Primary Energy Exports,float64
Primary Energy Net Imports,float64
Primary Energy Stock Change and Other,float64
Total Fossil Fuels Consumption,float64


In [35]:
data = energy.copy()

<div style="font-family: Arial, sans-serif; line-height: 1.6; color: #333; padding: 20px; background-color: #f4f4f9; border-radius: 8px; border: 1px solid #ddd;">
  <h2 style="color: #2c3e50; font-size: 24px;">Feature Selection and Model Evaluation Process</h2>

  <p style="font-size: 16px; color: #555;">In this section, we perform feature selection using different methods and evaluate the performance of an XGBoost regressor model on the selected features. The key steps are outlined below:</p>

  <ul style="font-size: 16px; color: #555;">
    <li><strong>Step 1:</strong> Select features using various methods: <strong>SelectKBest</strong>, <strong>PCA</strong>, and <strong>RFE</strong>.</li>
    <li><strong>Step 2:</strong> Train an XGBoost regressor model on the selected features.</li>
    <li><strong>Step 3:</strong> Evaluate the model performance using metrics such as MAE, MSE, R2, and AUC.</li>
  </ul>

  <h3 style="color: #2c3e50; font-size: 20px;">1. SelectKBest (Using F-statistic)</h3>
  <p style="font-size: 16px; color: #555;">SelectKBest is used to select the top features based on the F-statistic. The features are selected by computing their relationship with the target variable. In this case, we are using the 'Total Fossil Fuels Consumption' target for scoring.</p>

  <h3 style="color: #2c3e50; font-size: 20px;">2. PCA (Principal Component Analysis)</h3>
  <p style="font-size: 16px; color: #555;">Principal Component Analysis (PCA) is applied to reduce dimensionality while retaining 95% of the variance in the data. This allows us to project the data onto a smaller number of components, thereby simplifying the model without sacrificing much information.</p>

  <h3 style="color: #2c3e50; font-size: 20px;">3. Recursive Feature Elimination (RFE)</h3>
  <p style="font-size: 16px; color: #555;">RFE is used to recursively eliminate features and select the top 5 features based on an estimator, in this case, the XGBoost regressor. The process involves fitting the model, ranking features by their importance, and selecting the most relevant ones.</p>

  <h3 style="color: #2c3e50; font-size: 20px;">Training and Evaluating XGBoost Model</h3>
  <p style="font-size: 16px; color: #555;">After selecting the features, we train an XGBoost regressor model on the selected features. The model's performance is then evaluated using the following metrics:</p>
  <ul style="font-size: 16px; color: #555;">
    <li><strong>MAE:</strong> Mean Absolute Error</li>
    <li><strong>MSE:</strong> Mean Squared Error</li>
    <li><strong>R2:</strong> R-squared</li>
    <li><strong>AUC:</strong> Area Under the ROC Curve (for binary classification)</li>
  </ul>

  <h3 style="color: #


In [36]:
features = [
    'Total Fossil Fuels Production',
    'Nuclear Electric Power Production',
    'Total Renewable Energy Production',
    'Total Primary Energy Production',
    'Primary Energy Imports',
    'Primary Energy Exports',
    'Primary Energy Net Imports',
    'Primary Energy Stock Change and Other'
]
targets = [
    'Total Fossil Fuels Consumption',
    'Nuclear Electric Power Consumption',
    'Total Renewable Energy Consumption',
    'Total Primary Energy Consumption'
]

X = data[features]
y = data[targets]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# 1. SelectKBest (using F-statistic)
selector_kbest = SelectKBest(score_func=f_regression, k='all')  # Adjust k as needed
X_kbest = selector_kbest.fit_transform(X_scaled, y['Total Fossil Fuels Consumption'])  # Using one target for scoring
selected_features_kbest = np.array(features)[selector_kbest.get_support()]

# 2. PCA (Principal Component Analysis)
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_pca = pca.fit_transform(X_scaled)

# 3. Recursive Feature Elimination (RFE)
model = XGBRegressor(objective='reg:squarederror', random_state=42)  # Model for RFE
rfe = RFE(estimator=model, n_features_to_select=5)  # Select top 5 features
X_rfe = rfe.fit_transform(X_scaled, y['Total Fossil Fuels Consumption'])  # Using one target for evaluation
selected_features_rfe = np.array(features)[rfe.get_support()]

# Function to train XGBoost and evaluate
def train_and_evaluate_XGB(X_selected, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Train an XGBoost model
    model = XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate model performance
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # AUC: Treating the regression output as probabilities (threshold at median)
    y_pred_binary = (y_pred > y_test.median()).astype(int)  # Convert to binary for AUC
    auc = roc_auc_score((y_test > y_test.median()).astype(int), y_pred_binary)

    return mae, mse, r2, auc

# Evaluate models for each feature selection method
results = {}

# For SelectKBest
X_kbest_selected = X_scaled[:, selector_kbest.get_support()]
for target in targets:
    mae, mse, r2, auc = train_and_evaluate_XGB(X_kbest_selected, y[target])
    results[f"SelectKBest - {target}"] = {"MAE": mae, "MSE": mse, "R2": r2, "AUC": auc}

# For PCA
for target in targets:
    mae, mse, r2, auc = train_and_evaluate_XGB(X_pca, y[target])
    results[f"PCA - {target}"] = {"MAE": mae, "MSE": mse, "R2": r2, "AUC": auc}

# For RFE
X_rfe_selected = X_scaled[:, rfe.get_support()]
for target in targets:
    mae, mse, r2, auc = train_and_evaluate_XGB(X_rfe_selected, y[target])
    results[f"RFE - {target}"] = {"MAE": mae, "MSE": mse, "R2": r2, "AUC": auc}

# Output results
print("Model Performance Metrics:")
for method_target, metrics in results.items():
    print(f"\n{method_target}:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")


Model Performance Metrics:

SelectKBest - Total Fossil Fuels Consumption:
MAE: 0.1152
MSE: 0.0240
R2: 0.9461
AUC: 0.9113

SelectKBest - Nuclear Electric Power Consumption:
MAE: 0.0026
MSE: 0.0000
R2: 0.9997
AUC: 0.9919

SelectKBest - Total Renewable Energy Consumption:
MAE: 0.0039
MSE: 0.0000
R2: 0.9982
AUC: 0.9919

SelectKBest - Total Primary Energy Consumption:
MAE: 0.1427
MSE: 0.0331
R2: 0.9571
AUC: 0.9274

PCA - Total Fossil Fuels Consumption:
MAE: 0.2027
MSE: 0.0704
R2: 0.8420
AUC: 0.8790

PCA - Nuclear Electric Power Consumption:
MAE: 0.0359
MSE: 0.0025
R2: 0.9384
AUC: 0.9355

PCA - Total Renewable Energy Consumption:
MAE: 0.0210
MSE: 0.0007
R2: 0.9662
AUC: 0.8710

PCA - Total Primary Energy Consumption:
MAE: 0.1890
MSE: 0.0630
R2: 0.9184
AUC: 0.9274

RFE - Total Fossil Fuels Consumption:
MAE: 0.1122
MSE: 0.0230
R2: 0.9483
AUC: 0.9274

RFE - Nuclear Electric Power Consumption:
MAE: 0.0355
MSE: 0.0026
R2: 0.9355
AUC: 0.9435

RFE - Total Renewable Energy Consumption:
MAE: 0.0195
MS

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Function to train Linear Regression and evaluate
def train_and_evaluate_linear_regression(X_selected, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Train a Linear Regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate model performance
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # AUC: Treating the regression output as probabilities (threshold at median)
    y_pred_binary = (y_pred > y_test.median()).astype(int)  # Convert to binary for AUC
    auc = roc_auc_score((y_test > y_test.median()).astype(int), y_pred_binary)

    return mae, mse, r2, auc

# Evaluate models for each feature selection method
results = {}

# For SelectKBest
X_kbest_selected = X_scaled[:, selector_kbest.get_support()]
for target in targets:
    mae, mse, r2, auc = train_and_evaluate_linear_regression(X_kbest_selected, y[target])
    results[f"SelectKBest - {target}"] = {"MAE": mae, "MSE": mse, "R2": r2, "AUC": auc}

# For PCA
for target in targets:
    mae, mse, r2, auc = train_and_evaluate_linear_regression(X_pca, y[target])
    results[f"PCA - {target}"] = {"MAE": mae, "MSE": mse, "R2": r2, "AUC": auc}

# For RFE
X_rfe_selected = X_scaled[:, rfe.get_support()]
for target in targets:
    mae, mse, r2, auc = train_and_evaluate_linear_regression(X_rfe_selected, y[target])
    results[f"RFE - {target}"] = {"MAE": mae, "MSE": mse, "R2": r2, "AUC": auc}

# Output results
print("Model Performance Metrics with Linear Regression:")
for method_target, metrics in results.items():
    print(f"\n{method_target}:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")


In [37]:
data = energy.copy()

In [38]:
data.dtypes

,0
Month,datetime64[ns]
Total Fossil Fuels Production,float64
Nuclear Electric Power Production,float64
Total Renewable Energy Production,float64
Total Primary Energy Production,float64
Primary Energy Imports,float64
Primary Energy Exports,float64
Primary Energy Net Imports,float64
Primary Energy Stock Change and Other,float64
Total Fossil Fuels Consumption,float64


<div style="font-family: Arial, sans-serif; line-height: 1.6; color: #333; padding: 20px; background-color: #f4f4f9; border-radius: 8px; border: 1px solid #ddd;">
  <h2 style="color: #2c3e50; font-size: 24px;">Feature Engineering: Creating Weather-related Features</h2>
  
  <p style="font-size: 16px; color: #555;">In this section, we perform feature engineering by creating new weather-related features based on the existing data. The steps followed are:</p>
  
  <ul style="font-size: 16px; color: #555;">
    <li><strong>Step 1:</strong> Extract the month from the 'Month' column and use it to categorize months into seasons.</li>
    <li><strong>Step 2:</strong> Create a new feature called 'Hot Weather' which is set to 1 for months between June and August (inclusive) and 0 otherwise.</li>
    <li><strong>Step 3:</strong> Create a new feature called 'Cold Weather' which is set to 1 for December, January, and February (inclusive) and 0 for the rest of the months.</li>
    <li><strong>Step 4:</strong> Drop the original 'Month' column as it is no longer needed for the analysis.</li>
  </ul>
  
  <h3 style="color: #2c3e50; font-size: 20px;">Weather Features:</h3>
  <p style="font-size: 16px; color: #555;">The two new features created are:</p>
  <ul style="font-size: 16px; color: #555;">
    <li><strong>Hot Weather:</strong> A binary feature (1 for hot months, 0 for others) to capture the seasonal impact of summer months (June-August).</li>
    <li><strong>Cold Weather:</strong> A binary feature (1 for cold months, 0 for others) to capture the seasonal impact of winter months (December-February).</li>
  </ul>

  <h3 style="color: #2c3e50; font-size: 20px;">Next Steps:</h3>
  <p style="font-size: 16px; color: #555;">Once the new weather-related features are created, the modified dataset is ready for use in further analysis or modeling.</p>
  
  <p style="font-size: 16px; color: #555;">The data now includes the features that can potentially help in identifying seasonal trends and their influence on energy consumption patterns.</p>
</div>


In [39]:
import numpy as np

data['Month'] = data['Month'].dt.month

data['Hot Weather'] = np.where((data['Month'] >= 6) & (data['Month'] <= 8), 1, 0)

data['Cold Weather'] = np.where((data['Month'] == 12) | (data['Month'] <= 2), 1, 0)

data = data.drop('Month', axis=1)




In [40]:
data.dtypes

,0
Total Fossil Fuels Production,float64
Nuclear Electric Power Production,float64
Total Renewable Energy Production,float64
Total Primary Energy Production,float64
Primary Energy Imports,float64
Primary Energy Exports,float64
Primary Energy Net Imports,float64
Primary Energy Stock Change and Other,float64
Total Fossil Fuels Consumption,float64
Nuclear Electric Power Consumption,float64


<div style="font-family: Arial, sans-serif; line-height: 1.6; color: #333; padding: 20px; background-color: #f4f4f9; border-radius: 8px; border: 1px solid #ddd;">
  <h2 style="color: #2c3e50; font-size: 24px;">Feature Engineering and Model Evaluation for Weather Prediction</h2>
  
  <p style="font-size: 16px; color: #555;">In this section, we perform feature engineering and evaluate machine learning models for predicting weather-related features (Hot and Cold Weather) based on various energy-related features. The workflow includes:</p>
  
  <ul style="font-size: 16px; color: #555;">
    <li><strong>Feature Engineering:</strong> We use energy-related features to predict weather conditions.</li>
    <li><strong>Feature Scaling:</strong> Standard scaling is applied to the features to ensure uniformity and improve model performance.</li>
  </ul>
  
  <h3 style="color: #2c3e50; font-size: 20px;">Feature Selection Methods:</h3>
  <p style="font-size: 16px; color: #555;">We apply three different feature selection techniques:</p>
  <ul style="font-size: 16px; color: #555;">
    <li><strong>SelectKBest (ANOVA F-statistic):</strong> Selects the top features based on their statistical significance with respect to the target ('Hot Weather').</li>
    <li><strong>PCA (Principal Component Analysis):</strong> Reduces the dimensionality of the feature space, retaining 95% of the variance.</li>
    <li><strong>Recursive Feature Elimination (RFE):</strong> Selects the top 5 features that contribute most to the prediction of the target ('Hot Weather') using an XGBoost model.</li>
  </ul>
  
  <h3 style="color: #2c3e50; font-size: 20px;">Model Evaluation:</h3>
  <p style="font-size: 16px; color: #555;">After selecting the features, we train and evaluate an XGBoost classifier model using each of the feature sets. The evaluation metrics used are:</p>
  <ul style="font-size: 16px; color: #555;">
    <li><strong>Accuracy:</strong> Measures the proportion of correctly classified instances.</li>
    <li><strong>AUC (Area Under the ROC Curve):</strong> Evaluates the ability of the model to distinguish between the classes.</li>
    <li><strong>Classification Report:</strong> Provides precision, recall, and F1-score for the classification task.</li>
  </ul>
  
  <h3 style="color: #2c3e50; font-size: 20px;">Results:</h3>
  <p style="font-size: 16px; color: #555;">The performance of the models for each feature selection method is reported. For each method, the accuracy, AUC, and detailed classification report are provided.</p>
  
  <p style="font-size: 16px; color: #555;">This process helps us assess the impact of different feature selection techniques and determine which approach yields the best performance for predicting weather conditions based on energy data.</p>
</div>


In [41]:
features = [
    'Total Fossil Fuels Production',
    'Nuclear Electric Power Production',
    'Total Renewable Energy Production',
    'Total Primary Energy Production',
    'Primary Energy Imports',
    'Primary Energy Exports',
    'Primary Energy Net Imports',
    'Primary Energy Stock Change and Other',
    'Total Fossil Fuels Consumption',
    'Nuclear Electric Power Consumption',
    'Total Renewable Energy Consumption',
    'Total Primary Energy Consumption'
]
targets = ['Hot Weather', 'Cold Weather']

X = data[features]
y = data[targets]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# 1. SelectKBest (using ANOVA F-statistic)
selector_kbest = SelectKBest(score_func=f_classif, k='all')  # Choose top k features, here 'all'
X_kbest = selector_kbest.fit_transform(X_scaled, y['Hot Weather'])  # Use Hot Weather for evaluation
selected_features_kbest = np.array(features)[selector_kbest.get_support()]

# 2. PCA (Principal Component Analysis)
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_pca = pca.fit_transform(X_scaled)

# 3. Recursive Feature Elimination (RFE)
model = XGBClassifier(random_state=42)  # Use XGBoost model for RFE
rfe = RFE(estimator=model, n_features_to_select=5)  # Select top 5 features
X_rfe = rfe.fit_transform(X_scaled, y['Hot Weather'])  # Using Hot Weather for evaluation
selected_features_rfe = np.array(features)[rfe.get_support()]

# Train and evaluate classification model

def train_and_evaluate_model(X_selected, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Initialize the classifier (you can use any model like Logistic Regression, Random Forest, or XGBoost)
    model = XGBClassifier(random_state=42)  # XGBoost Classifier

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for AUC

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    return accuracy, report, auc

# Results dictionary to store performance
results = {}

# For SelectKBest
X_kbest_selected = X_scaled[:, selector_kbest.get_support()]
accuracy, report, auc = train_and_evaluate_model(X_kbest_selected, y['Hot Weather'])
results['SelectKBest - Hot Weather'] = {'Accuracy': accuracy, 'Report': report, 'AUC': auc}

# For PCA
accuracy, report, auc = train_and_evaluate_model(X_pca, y['Hot Weather'])
results['PCA - Hot Weather'] = {'Accuracy': accuracy, 'Report': report, 'AUC': auc}

# For RFE
X_rfe_selected = X_scaled[:, rfe.get_support()]
accuracy, report, auc = train_and_evaluate_model(X_rfe_selected, y['Hot Weather'])
results['RFE - Hot Weather'] = {'Accuracy': accuracy, 'Report': report, 'AUC': auc}

# Output results
print("Model Evaluation Metrics:")
for method_target, metrics in results.items():
    print(f"\n{method_target}:")
    print(f"Accuracy: {metrics['Accuracy']:.4f}")
    print(f"AUC: {metrics['AUC']:.4f}")
    print(f"Classification Report:\n{metrics['Report']}")


Model Evaluation Metrics:

SelectKBest - Hot Weather:
Accuracy: 0.7984
AUC: 0.8556
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87        90
           1       0.67      0.53      0.59        34

    accuracy                           0.80       124
   macro avg       0.75      0.71      0.73       124
weighted avg       0.79      0.80      0.79       124


PCA - Hot Weather:
Accuracy: 0.7339
AUC: 0.8281
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.91      0.83        90
           1       0.53      0.26      0.35        34

    accuracy                           0.73       124
   macro avg       0.65      0.59      0.59       124
weighted avg       0.70      0.73      0.70       124


RFE - Hot Weather:
Accuracy: 0.7903
AUC: 0.8539
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.90      0.